In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import datetime
import functools

from typing import List, Union, Optional, Dict

import src.featurelib as fl

from datetime import datetime, timedelta, date

import re
import sklearn.base as skbase

/Users/19587293/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be remove

### Get data

In [2]:
receipts = dd.read_parquet('test/data/receipts.parquet/')

In [3]:
TABLES = {
    'receipts': receipts,
#     'campaigns': campaigns,
#     'client_profile': client_profile,
#     'products': products,
#     'purchases': purchases,
}

In [4]:
engine = fl.Engine(tables=TABLES)

In [5]:
receipts.head()

,__null_dask_index__,client_id,transaction_id,trn_sum_from_red,trn_sum_from_iss,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,transaction_datetime
__null_dask_index__,,,,,,,,,,,,
0,0,5368,572976,0.0,427.0,2.1,0.0,0.0,0.0,427.00,2762,2018-11-27 15:52:50
1,1,5368,1095593,0.0,438.0,2.1,0.0,0.0,0.0,437.00,3395,2019-02-23 10:22:23
2,2,5368,1843577,0.0,1890.0,14.7,0.0,0.0,0.0,1889.00,3395,2019-02-23 10:21:29
3,3,5368,2389159,0.0,3274.0,32.7,0.0,0.0,0.0,3273.00,3395,2018-12-15 10:41:28
4,4,5368,3766070,0.0,1585.0,11.7,0.0,0.0,0.0,1585.43,3395,2019-02-08 08:41:18


In [6]:
test_output = pd.read_csv('./test/dataset_mini.csv')

In [7]:
test_output.head()

,client_id,purchases_count_dw5__30d,purchases_count_dw4__30d,purchases_count_dw1__30d,purchases_count_dw6__30d,purchases_count_dw0__30d,purchases_count_dw3__30d,purchases_count_dw2__30d,favourite_store_id__30d,age,gender,treatment_flg,target_purchases_sum,target_purchases_count,target_campaign_points_spent,weekend_purchases_ratio__30d,target_profit,gender__mte__target_profit,gender__mte__target_purchases_count
0,5368,2.0,2.0,1.0,1.0,0.0,0.0,0.0,3395,62,F,0,2813.45,3,0.0,0.50,562.690,106.477263,1.973684
1,11882,0.0,1.0,1.0,3.0,1.0,0.0,0.0,125,69,F,0,307.00,2,0.0,0.50,61.400,119.669105,2.000000
2,15640,2.0,3.0,0.0,1.0,0.0,0.0,0.0,11168,52,U,0,137.97,1,0.0,0.50,27.594,131.451902,1.097561
3,17317,1.0,2.0,0.0,0.0,1.0,1.0,0.0,12767,66,F,1,284.00,1,-0.0,0.20,55.300,119.829632,2.026316
4,25169,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2200,63,F,0,111.00,1,0.0,0.25,22.200,120.700684,2.026316


### FavouriteStoreCalcer

In [8]:
class FavouriteStoreCalcer(fl.DateFeatureCalcer):
    name = 'favourite_store'
    keys = ['client_id']

    def __init__(self, delta: int, **kwargs):
        self.delta = delta
        super().__init__(**kwargs)
        
    def compute(self) -> dd.DataFrame:
        delta = timedelta(days=self.delta)
        date_to = pd.Timestamp(self.date_to)
        input_data = self.engine.get_table('receipts')
        col_name = "favourite_store_id__{delta}d".format(delta=self.delta)
            
        data = input_data.loc[(input_data['transaction_datetime'] < date_to) & \
              (input_data['transaction_datetime'] >= date_to - delta)][
            ['client_id', 'store_id', 'transaction_datetime']
        ].reset_index(drop=True)
        data = data.compute()
        
        data = pd.pivot_table(
            data,
            values = 'transaction_datetime',
            index = ['client_id', 'store_id'],
            aggfunc = len
        ).sort_values(['client_id', 'transaction_datetime', 'store_id'],
                      ascending = [True, False, False]).reset_index()
        
        data = data.groupby(by='client_id').first().reset_index().drop(columns=['transaction_datetime'])
        col_name = "favourite_store_id__{delta}d".format(delta=self.delta)
        data = data.rename(columns={'store_id': col_name})
        return dd.from_pandas(data, npartitions=1)

In [9]:
calcer = FavouriteStoreCalcer(
    engine=engine,
    delta=30,
    date_to=date(2019, 3, 19)
    )

In [10]:
output = calcer.compute().compute()

In [11]:
sum(output['favourite_store_id__30d'] == test_output['favourite_store_id__30d'])

100

### DayOfWeekReceiptsCalcer

In [12]:
class DayOfWeekReceiptsCalcer(fl.DateFeatureCalcer):
    name = 'day_of_week_receipts'
    keys = ['client_id']

    def __init__(self, delta: int, **kwargs):
        self.delta = delta
        super().__init__(**kwargs)
        
    def compute(self) -> dd.DataFrame:
        # columns:
        # client_id
        # purchases_count_dw{day}__{delta}d
        # число покупок (чеков) в день недели с номером day (от 0 до 6, пн = 0),
        # за delta дней до даты date_to (не включая день date_to)
        delta = timedelta(days=self.delta)
        date_to = pd.Timestamp(self.date_to)
        input_data = self.engine.get_table('receipts')
        
        data = input_data.loc[(input_data['transaction_datetime'] < date_to) & \
              (input_data['transaction_datetime'] >= date_to - delta)][
            ['client_id', 'transaction_datetime', 'transaction_id']
        ].reset_index(drop=True)
        data = data.compute()
        data['day_of_week'] = [date.weekday() for date in data.loc[:, ('transaction_datetime')]]
        data = pd.pivot_table(
                    data,
                    columns = 'day_of_week',
                    values = 'transaction_id',
                    index = ['client_id'],
                    aggfunc = 'count',
                    dropna = True
                )
        data = data.fillna(value=0)
        rename_dict = {}
        for i in data.columns.tolist():
            col_name = "purchases_count_dw{day}__{delta}d".format(day=i, delta=self.delta)
            rename_dict[i] = col_name
        data = data.rename(columns=rename_dict)  
        data = data.reset_index()
        data.columns.name = ''
        return dd.from_pandas(data, npartitions=1)

In [13]:
calcer = DayOfWeekReceiptsCalcer(
    engine=engine,
    delta=30,
    date_to=date(2019, 3, 19)
    )

In [14]:
output = calcer.compute().compute()

In [15]:
output.columns

Index(['client_id', 'purchases_count_dw0__30d', 'purchases_count_dw1__30d',
       'purchases_count_dw2__30d', 'purchases_count_dw3__30d',
       'purchases_count_dw4__30d', 'purchases_count_dw5__30d',
       'purchases_count_dw6__30d'],
      dtype='object', name='')

In [16]:
for i in output.columns.tolist():
    print(i, sum(output[i] == test_output[i]))

client_id 100
purchases_count_dw0__30d 100
purchases_count_dw1__30d 100
purchases_count_dw2__30d 100
purchases_count_dw3__30d 100
purchases_count_dw4__30d 100
purchases_count_dw5__30d 100
purchases_count_dw6__30d 100


### ExpressionTransformer

In [17]:
class ExpressionTransformer(skbase.BaseEstimator, skbase.TransformerMixin):
    
    def __init__(self, expression, col_result):
        super().__init__()
        self.expression = expression
        self.col_result = col_result
        
    def fit(self, *args, **kwargs):
        return self

    def transform(self, data: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
        result = eval(self.expression.format(d='data'))
                
        data[self.col_result] = result
        return data

In [18]:
exp = "({d}['purchases_count_dw2__30d'] + {d}['purchases_count_dw3__30d'] + {d}['purchases_count_dw1__30d'])"

out_data = ExpressionTransformer(exp, 'sum_result').transform(output)

out_data.head()

,client_id,purchases_count_dw0__30d,purchases_count_dw1__30d,purchases_count_dw2__30d,purchases_count_dw3__30d,purchases_count_dw4__30d,purchases_count_dw5__30d,purchases_count_dw6__30d,sum_result
0,5368,0.0,1.0,0.0,0.0,2.0,2.0,1.0,1.0
1,11882,1.0,1.0,0.0,0.0,1.0,0.0,3.0,1.0
2,15640,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.0
3,17317,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0
4,25169,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2.0


### LOOMeanTargetEncoder

In [19]:
class LOOMeanTargetEncoder(skbase.BaseEstimator, skbase.TransformerMixin):
    
    def __init__(
        self,
        col_categorical: str, # название колонки с категориальным признаком, который нужно закодировать
        col_target: str, # название колонки, из которой берется значение целевой переменной
        col_result: str, # название колонки, в которой будет сохранен результат
    ):
        super().__init__()
        self.col_categorical = col_categorical
        self.col_target = col_target
        self.col_result = col_result
        
        self.encoder = LeaveOneOutEncoder()

    def fit(self, data: pd.DataFrame, *args, **kwargs):
        if self.col_target in data.columns:
            self.encoder.fit(X=data[self.col_categorical], y=data[self.col_target])
        else:
            self.encoder = self.encoder.fit(X=data[self.col_categorical], y=None)
        return self
    
    def transform(self, data: pd.DataFrame, *args, **kwargs) -> pd.DataFrame:
        if self.col_target in data.columns:
            result = self.encoder.transform(X=data[self.col_categorical], y=data[self.col_target])
        else:
            result = self.encoder.transform(X=data[self.col_categorical], y=None)
        
        data[self.col_result] = result
        return data